In [203]:
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage, Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import requests
from io import BytesIO
from langchain_community.chat_models import ChatOllama

In [204]:
# CSV 파일 로드(재료 영양소)
food_nutrition1 = pd.read_csv("../dataset/FINAL_FOOD_DATASET/FOOD-DATA-GROUP1.csv", encoding='CP949')
food_nutrition2 = pd.read_csv("../dataset/FINAL_FOOD_DATASET/FOOD-DATA-GROUP2.csv", encoding='CP949')
food_nutrition3 = pd.read_csv("../dataset/FINAL_FOOD_DATASET/FOOD-DATA-GROUP3.csv", encoding='CP949')
food_nutrition4 = pd.read_csv("../dataset/FINAL_FOOD_DATASET/FOOD-DATA-GROUP4.csv", encoding='CP949')
food_nutrition5 = pd.read_csv("../dataset/FINAL_FOOD_DATASET/FOOD-DATA-GROUP5.csv", encoding='CP949')
concatenated_food_nutrition = pd.concat([food_nutrition1, food_nutrition2, food_nutrition3, food_nutrition4, food_nutrition5], ignore_index=True)
concatenated_food_nutrition.tail()


,Unnamed: 0.1,Unnamed: 0,food,Caloric Value,Fat,Saturated Fats,Monounsaturated Fats,Polyunsaturated Fats,Carbohydrates,Sugars,...,Calcium,Copper,Iron,Magnesium,Manganese,Phosphorus,Potassium,Selenium,Zinc,Nutrition Density
2390,717,717,jews ear,25,0.095,0.000,0.000,0.000,6.7,0.000,...,15.8,0.4,0.600,24.800,0.040,13.900,42.6,0.034,0.700,24.249
2391,718,718,enoki mushrooms,1,0.099,0.027,0.000,0.010,0.2,0.034,...,0.0,0.0,0.099,0.034,0.500,0.015,3.2,10.800,0.096,0.433
2392,719,719,morel mushrooms,4,0.070,0.056,0.031,0.007,0.7,0.096,...,0.0,5.5,0.056,1.600,2.500,0.060,25.0,53.000,0.069,1.727
2393,720,720,portabella mushrooms raw,19,0.300,0.036,0.016,0.100,3.3,2.200,...,2.6,0.2,0.300,0.000,0.092,92.900,313.0,0.015,0.500,9.400
2394,721,721,oyster mushroom,5,0.035,0.016,0.039,0.099,0.9,0.200,...,0.0,0.5,0.008,0.200,2.700,0.055,18.0,63.000,0.048,1.804


In [205]:
# CSV 파일 로드(음식 재료)
food_ingredient = pd.read_csv("../dataset/Food_Ingredients_Dataset.csv")
food_ingredient.tail()

,Unnamed: 0,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
13496,13496,Brownie Pudding Cake,"['1 cup all-purpose flour', '2/3 cup unsweeten...",Preheat the oven to 350°F. Into a bowl sift to...,brownie-pudding-cake-14408,"['1 cup all-purpose flour', '2/3 cup unsweeten..."
13497,13497,Israeli Couscous with Roasted Butternut Squash...,"['1 preserved lemon', '1 1/2 pound butternut s...",Preheat oven to 475°F.\nHalve lemons and scoop...,israeli-couscous-with-roasted-butternut-squash...,"['1 preserved lemon', '1 1/2 pound butternut s..."
13498,13498,Rice with Soy-Glazed Bonito Flakes and Sesame ...,['Leftover katsuo bushi (dried bonito flakes) ...,"If using katsuo bushi flakes from package, moi...",rice-with-soy-glazed-bonito-flakes-and-sesame-...,['Leftover katsuo bushi (dried bonito flakes) ...
13499,13499,Spanakopita,['1 stick (1/2 cup) plus 1 tablespoon unsalted...,Melt 1 tablespoon butter in a 12-inch heavy sk...,spanakopita-107344,['1 stick (1/2 cup) plus 1 tablespoon unsalted...
13500,13500,"Mexican Poblano, Spinach, and Black Bean ""Lasa...",['12 medium to large fresh poblano chiles (2 1...,Lay 4 chiles on their sides on racks of gas bu...,mexican-poblano-spinach-and-black-bean-lasagne...,['12 medium to large fresh poblano chiles (2 1...


In [206]:
# 텍스트 분할
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts_nutrition = text_splitter.split_text("\n".join(concatenated_food_nutrition.to_string()))
texts_ingredient = text_splitter.split_text("\n".join(food_ingredient.to_string()))

Created a chunk of size 38019, which is longer than the specified 1000
Created a chunk of size 38020, which is longer than the specified 1000
Created a chunk of size 38020, which is longer than the specified 1000
Created a chunk of size 38020, which is longer than the specified 1000
Created a chunk of size 38020, which is longer than the specified 1000
Created a chunk of size 38020, which is longer than the specified 1000
Created a chunk of size 38020, which is longer than the specified 1000
Created a chunk of size 38020, which is longer than the specified 1000
Created a chunk of size 38020, which is longer than the specified 1000
Created a chunk of size 38020, which is longer than the specified 1000
Created a chunk of size 38020, which is longer than the specified 1000
Created a chunk of size 38020, which is longer than the specified 1000
Created a chunk of size 38020, which is longer than the specified 1000
Created a chunk of size 38020, which is longer than the specified 1000
Create

In [207]:
# 텍스트 데이터를 벡터화
# texts_nutrition_vectors = concatenated_food_nutrition
# texts_ingredient_vectors = food_ingredient

# 텍스트를 Document 객체로 변환 (FAISS가 요구하는 형식)
documents_nutrition = [Document(page_content=text) for text in texts_nutrition]
documents_ingredient = [Document(page_content=text) for text in texts_ingredient]


# 임베딩 초기화
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 각 텍스트에 대해 임베딩 생성
embeddings_nutrition = embedding_model.embed_documents(texts_nutrition)
embeddings_ingredient = embedding_model.embed_documents(texts_ingredient)
# 임베딩 모델 초기화
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v2")
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [208]:
# 벡터 데이터베이스 생성
all_dataset = documents_nutrition + documents_ingredient
# all_embedding = embeddings_nutrition_vectors + embeddings_ingredient_vectors
vectorstore = FAISS.from_documents(all_dataset, embedding_model)


In [209]:
# 벡터 데이터베이스 저장
vectorstore.save_local("../llm")

In [231]:
# Ollama 서버 설정
OLLAMA_SERVER = "http://localhost:11434"
MODEL_NAME = "gemma2"
llm = ChatOllama(model=MODEL_NAME, base_url=OLLAMA_SERVER, temperature=0.1)

In [232]:
# ConversationalRetrievalChain 설정
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(search_kwargs={"k": 1}),
    return_source_documents=True,
    verbose=False
)

In [233]:
# TensorFlow MobileNetV2 모델 로드
model = tf.keras.applications.MobileNetV2(weights="imagenet")

In [234]:
# 음식 정보 추출 함수 (재료, 영양성분)
def get_food_info_from_vectorstore(food_name, vectorstore):
    try:
        # food_name을 쿼리로 변환
        query = f"Tell me about {food_name}"

        # 쿼리를 벡터화
        query_vector = vectorstore.embeddings.embed_query(query)

        # FAISS에서 검색
        D, I = vectorstore.index.search(np.array([query_vector]).astype('float32'), 1)  # 상위 1개 검색

        # 가장 유사한 음식 정보 가져오기
        if I[0][0] != -1:  # 유효한 결과가 있을 경우
            # vectorstore에 있는 음식 정보 가져오기 (food_name을 통해)
            matched_food = vectorstore.docs[I[0][0]]
            return matched_food
        else:
            return {"error": "음식 정보를 찾을 수 없습니다."}
    except Exception as e:
        return {"error": f"Error: {e}"}

In [235]:
# Ollama를 사용해 음식 설명 생성
def get_food_description_with_langchain(food_name):
    """
    LangChain ChatOllama를 사용하여 음식 설명 생성
    """
    try:
        response = llm([HumanMessage(content=f"Tell me about {food_name}.")])
        if isinstance(response, AIMessage):
            return response.content
        return f"Unexpected response: {response}"
    except Exception as e:
        return f"Failed to retrieve description: {e}"

In [236]:
# 음식 재료와 영양 정보 찾기
def get_food_nutrition_and_ingredient(predicted_ingredient):
    # Step 1: 영양소 정보 가져오기
    matching_nutrition_row = concatenated_food_nutrition[concatenated_food_nutrition['food'] == predicted_ingredient]
    
    if not matching_nutrition_row.empty:
        nutrition_info = matching_nutrition_row.iloc[0].to_dict()  # 첫 번째 행을 딕셔너리로 변환
    else:
        nutrition_info = {"error": "No matching food found in nutrition data"}
    
    # Step 2: 재료 정보 가져오기
    matching_ingredient_row = food_ingredient[food_ingredient['Title'] == predicted_ingredient]
    
    if not matching_ingredient_row.empty:
        ingredient_info = matching_ingredient_row.iloc[0].to_dict()  # 첫 번째 행을 딕셔너리로 변환
    else:
        ingredient_info = {"error": "No matching ingredient found"}
    
    return nutrition_info, ingredient_info


In [237]:
def filter_dataframes_by_food_name(food_name, concatenated_food_nutrition, food_ingredient):
    """
    특정 food_name이 포함된 데이터를 각 데이터프레임에서 필터링하고,
    Unnamed로 시작하는 컬럼은 제외합니다.
    """
    # "Unnamed" 컬럼 제거
    nutrition_filtered = concatenated_food_nutrition.loc[
        concatenated_food_nutrition["food"].str.contains(food_name, case=False, na=False)
    ]
    nutrition_filtered = nutrition_filtered.loc[:, ~nutrition_filtered.columns.str.startswith("Unnamed")]

    ingredient_filtered = food_ingredient.loc[
        food_ingredient["Title"].str.contains(food_name, case=False, na=False)
    ]
    ingredient_filtered = ingredient_filtered.loc[:, ~ingredient_filtered.columns.str.startswith("Unnamed")]

    return nutrition_filtered, ingredient_filtered

In [238]:
# 빈 데이터 프레임 대체 문구
def handle_empty_dataframe(df, no_info_message="No info"):
    """
    빈 데이터프레임을 no_info_message로 채워진 데이터프레임으로 변환
    """
    if df.empty:
        columns = ["Column1"] if df.columns.empty else df.columns  # 기본 컬럼 이름 생성
        return pd.DataFrame({col: [no_info_message] for col in columns})
    return df

In [239]:
# 이미지 예측 함수

def predict_image_with_description(image_url):
    """
    이미지 URL을 받아 음식 예측과 Ollama 설명, 데이터 필터링 결과를 반환
    """
    try:
        # 이미지 처리
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).resize((224, 224))

        image_array = tf.keras.preprocessing.image.img_to_array(image)
        image_array = tf.expand_dims(image_array, axis=0)
        image_array = tf.keras.applications.mobilenet_v2.preprocess_input(image_array)

        # 이미지 예측
        predictions = model.predict(image_array)
        decoded_predictions = tf.keras.applications.mobilenet_v2.decode_predictions(predictions, top=3)[0]
        result = {label: float(prob) for (_, label, prob) in decoded_predictions}

        # 가장 높은 확률의 예측값
        top_food = decoded_predictions[0][1]

        # Ollama 설명 생성
        description = get_food_description_with_langchain(top_food)

        # 데이터프레임 필터링
        nutrition_filtered, ingredient_filtered = filter_dataframes_by_food_name(
            top_food, concatenated_food_nutrition, food_ingredient
        )

        # 빈 데이터프레임 처리 (영양 정보와 재료 정보에 각각 처리)
        nutrition_output = handle_empty_dataframe(nutrition_filtered)
        ingredient_output = handle_empty_dataframe(ingredient_filtered)

        # 결과 반환 (표 형식으로 변환)
        return result, description, nutrition_output, ingredient_output
    
    except Exception as e:
        return {"error": 1.0}, f"Error: {e}", "No info", "No info"

In [240]:
# Gradio 인터페이스 설정
with gr.Blocks() as iface:
    # 제목 및 설명
    gr.Markdown("# 음식 재료 및 영양 정보")
    gr.Markdown("이미지 URL을 입력하면 해당 음식에 대한 정보를 출력합니다.")

    # 입력 필드, 출력 필드 (예측 결과)
    with gr.Row():
        with gr.Column():
            image_url_input = gr.Textbox(label="이미지 URL 입력")
        with gr.Column():
            prediction_label = gr.Label(num_top_classes=3, label="예측 결과")
    # 출력 필드 (설명 및 표)
    with gr.Row():
        food_description_output = gr.Textbox(label="음식 설명", interactive=False)
    
    with gr.Row():
        nutrition_info_output = gr.DataFrame(label="영양 정보", interactive=False)
    
    with gr.Row():
        ingredient_info_output = gr.DataFrame(label="재료 정보", interactive=False)

    # 버튼으로 실행
    predict_button = gr.Button("예측 실행")
    predict_button.click(
        fn=predict_image_with_description,
        inputs=[image_url_input],
        outputs=[prediction_label, food_description_output, nutrition_info_output, ingredient_info_output],
    )


In [241]:
# 인터페이스 실행
iface.launch(server_port=7861, server_name="0.0.0.0", debug=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 21ms/step
Keyboard interruption in main thread... closing server.


In [230]:
iface.close()

Closing server running on port: 7861
